In [2]:
import numpy as np
import pandas as pd
import json

In [2]:
s = pd.Series([1, 3, 5, np.nan, 6, 8])
s

0    1.0
1    3.0
2    5.0
3    NaN
4    6.0
5    8.0
dtype: float64

In [ ]:


#https://api.nytimes.com/svc/search/v2/articlesearch.json?&pub_year=2020&api-key=nKBpzKBvOYipTxZf1T1NwCsVeXad5xoE

In [39]:

from pynytimes import NYTAPI
import urllib3, json, datetime
nyt = NYTAPI("nKBpzKBvOYipTxZf1T1NwCsVeXad5xoE", parse_dates=True)
articles = nyt.article_search(
    results = 5000,
    dates = {
        "begin": datetime.datetime(2020, 1, 31),
        "end": datetime.datetime(2020, 12, 30)
    },
    options = {
        "sources": [
            "New York Times",
        ],
        "news_desk": [
            "OpEd",
            "Editorial"
            
        ]
    }
)

/Users/joshs/opt/anaconda3/lib/python3.8/site-packages/pynytimes/api.py:630: UserWarning: Asking for a lot of results, because of rate limits it can take a while.
  warnings.warn(
/Users/joshs/opt/anaconda3/lib/python3.8/site-packages/pynytimes/api.py:637: UserWarning: Asking for more results then the API can provide, loading maximum results.
  warnings.warn(


In [40]:


text_file = open("articles.txt", "w")
for article in articles:
    n = text_file.write(article['lead_paragraph'] + " ")
text_file.close()
    

In [41]:
def split_into_words(any_chunk_of_text):
    lowercase_text = any_chunk_of_text.lower()
    split_words = re.split("\W+", lowercase_text) 
    return split_words

In [42]:

training_data = []
for file in files:
    text = open(file, encoding='utf-8').read()
    processed_text = little_mallet_wrapper.process_string(text, numbers='remove')
    training_data.append(processed_text)

NameError: name 'files' is not defined

In [43]:
import re
from collections import Counter
full_text = open("articles.txt", encoding="utf-8").read()
number_of_desired_words = 10
# Manipulate and Analyze File
stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours',
'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers',
 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves',
 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are',
 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does',
 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until',
 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into',
 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down',
 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here',
 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',
 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so',
 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now', 've', 'll', 'amp']


all_the_words = split_into_words(full_text)
meaningful_words = [word for word in all_the_words if word not in stopwords]
meaningful_words_tally = Counter(meaningful_words)
most_frequent_meaningful_words = meaningful_words_tally.most_common(number_of_desired_words)

In [44]:
most_frequent_meaningful_words

[('trump', 531),
 ('president', 362),
 ('one', 274),
 ('election', 227),
 ('new', 217),
 ('like', 207),
 ('people', 206),
 ('years', 194),
 ('biden', 192),
 ('donald', 190)]

In [53]:
import little_mallet_wrapper
import seaborn
import glob
from pathlib import Path
path_to_mallet = 'mallet-2.0.8bin/mallet'
num_topics = 5
training_data = []
for article in articles:
    processed_text = little_mallet_wrapper.process_string(article['lead_paragraph'], numbers='remove')
    training_data.append(processed_text)
    
    
little_mallet_wrapper.print_dataset_stats(training_data)
#Change to your desired output directory
output_directory_path = 'topic-model-output'

#No need to change anything below here
Path(f"{output_directory_path}").mkdir(parents=True, exist_ok=True)

path_to_training_data           = f"{output_directory_path}/training.txt"
path_to_formatted_training_data = f"{output_directory_path}/mallet.training"
path_to_model                   = f"{output_directory_path}/mallet.model.{str(num_topics)}"
path_to_topic_keys              = f"{output_directory_path}/mallet.topic_keys.{str(num_topics)}"
path_to_topic_distributions     = f"{output_directory_path}/{str(num_topics)}"
little_mallet_wrapper.import_data(path_to_mallet,
                path_to_training_data,
                path_to_formatted_training_data,
                training_data)

Number of Documents: 2010
Mean Number of Words per Document: 26.0
Vocabulary Size: 11537
Importing data...
Complete


In [55]:
little_mallet_wrapper.train_topic_model(path_to_mallet,
                      path_to_formatted_training_data,
                      path_to_model,
                      path_to_topic_keys,
                      path_to_topic_distributions,
                      num_topics)

Training topic model...
Complete


In [52]:
topics = little_mallet_wrapper.load_topic_keys(path_to_topic_keys)

for topic_number, topic in enumerate(topics):
    print(f"✨Topic {topic_number}✨\n\n{topic}\n")

FileNotFoundError: [Errno 2] No such file or directory: 'topic-model-output/mallet.topic_keys.5'